# This is the Final build for the Project

### There are 3 main stages in this Project and one auxiliary stage which is run to set up the project. 
First, the `final_project.ipynb` is run to populate the PostGresDB and do some data cleaning. 
Later this file is run.  

This file contains 3 phases:
Data Generation -> Graph Building -> Visualization


## Stage 1: Data Generation


<div class="alert alert-block alert-danger">
<b>Danger:</b> Run Stage 0 before starting this
</div>

### Input: Time Period, Poolsize
### Output: CSV / Pandas DF that contains data for the next cycle
* The CSV will contain these fields: 
    * id
    * tpep_pickup_datetime
    * tpep_dropoff_datetime
    * passenger_count
    * trip_distance
    * PULocationID
    * DOLocationID
